# Exercício MNIST com várias camadas

O objetivo deste notebook é treinar uma rede com várias camadas para o dataset MNIST. Para isso complete as partes de código sinalizadas que estão faltando.

## Importação das bibliotecas

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

## Funções de predição e acurácia

In [ ]:
def predict(model, inputs):
    outputs = model(Variable(inputs))
    _, predicts = torch.max(outputs, 1)

    return predicts.data.numpy()

def getAccuracy(model, inputs, targets):
    outputs = model(Variable(inputs))
    _, predicts = torch.max(outputs, 1)

    predicts = predicts.data.numpy()
    targets = targets.numpy()

    accuracy = (predicts == targets).mean()
    return accuracy

## Carregamento dos dados do MNIST

In [ ]:
dataset_dir = '/data/datasets/MNIST/'

# Transformara os dados em tensores no intervalo [0.0, 1.0] (Os dados serão normalizados)
data_transform = transforms.ToTensor()

# carrega o conjunto de treinamento e de teste
train_dataset = MNIST(dataset_dir, train=True, transform=data_transform)
test_dataset = MNIST(dataset_dir, train=False, transform=data_transform)

print("Tamanho do dataset de treino:", len(train_dataset))
print("Tamanho do dataset de teste: ", len(test_dataset))

print("\nDimensões dos dados das imagens:", train_dataset[0][0].size())
print("Tipo dos dados das imagens:     ", type(train_dataset[0][0]))
print("Tipo das classes das imagens:   ", type(train_dataset[0][1]))

## Organizando e normalizando os dados

Neste exemplo utilizaremos 500 amostras de treinamento e 100 amostras para testes.


In [ ]:
n_samples_train = 1000
n_samples_test  = 500

train_dataset.train_data   = train_dataset.train_data[:n_samples_train]
train_dataset.train_labels = train_dataset.train_labels[:n_samples_train]
test_dataset.test_data   = test_dataset.test_data[:n_samples_test]
test_dataset.test_labels = test_dataset.test_labels[:n_samples_test]

print('Amostras para treinamento:', len(train_dataset))
print('Amostras para validação:',   len(test_dataset))

## Visualizando os dados

In [ ]:
n_samples = 24

# cria um DataLoader temporario para pegar um batch de 'n_samples' imagens de treinamento
temp_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                              batch_size=n_samples,
                                              shuffle=True)

# pega um batch de imagens
image_batch, labels = next(iter(temp_dataloader))

# cria um grid com as imagens
grid = torchvision.utils.make_grid(image_batch, normalize=True, pad_value=1.0, padding=1)

plt.figure(figsize=(15, 10))
plt.imshow(grid.numpy().transpose(1, 2, 0))
plt.axis('off')
plt.show()

## Treinamento

### Exercício 1 - Criação dos DataLoaders

Crie DataLoaders de treino e validação com somente um batch de dados.

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset=, 
                                               batch_size=,
                                               shuffle=True)

test_dataloader = 

In [ ]:
# verificação do exercício 1
if len(train_dataloader) > 1:
    print('train_dataloader tem mais de 1 batch')
else:
    print('train_dataloader está correto')
    
if len(test_dataloader) > 1:
    print('train_dataloader tem mais de 1 batch')
else:
    print('train_dataloader está correto')

### Exercício 2 - Complete a criação da rede

In [ ]:
class Modelo(nn.Module):
    def __init__(self):
        super(Modelo, self).__init__()

    
    def forward(self, x):
        return x
    
model = Modelo()
print(model)

In [ ]:
#Verificação do exercicio 2
test_input  = torch.FloatTensor(3, 28, 28)
test_output = model(Variable(test_input))

if test_output.size(1) == 10 and test_output.size(0) == test_input.size(0):
    print('Rede está funcionando corretamente')
elif test_output.size(1) != 10:
    print('Saída da rede deve ter dimensão (n_amostras, 10), mas tem dimensão ({}, {})'.format(
        test_output.size(0), test_output.size(0)))
elif test_output.size(0) != test_input.size(0):
    print('Número de amostras na saída ({}) deve ser igual ao da entrada ({})'.format(
        test_output.size(0), test_input.size(0)))

### Inicialização dos parâmetros

In [ ]:
learningRate = 0.5

# Utilizaremos CrossEntropyLoss como função de perda
criterion = torch.nn.CrossEntropyLoss()

# Nosso otomizador será SDG
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

### Laço de treinamento dos pesos

In [ ]:
epochs = 1000

# Pega todas as imagens de uma vez
input_data, targets_data = next(iter(train_dataloader))
# Transforma em vetor
input_data = input_data.view(-1, 28*28)

losses = []

for i in range(epochs):
    
    # calcula a saída da operação linear
    output = model(Variable(input_data))

    # calcula a perda
    loss = criterion(output, Variable(targets_data))

    # zero, backpropag gradientes, ajusta parâmetros gradiente descendente
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    losses.append(loss.data[0])
    
print('Final loss:', loss.data[0])

## Visulizando gráfico de perda durante o treinamento

In [ ]:
plt.plot(losses)

## Avaliando a acurácia tanto no conjunto de treinamento como no conjunto de testes

In [ ]:
print('Training Accuracy: ', getAccuracy(model, input_data, targets_data))

test_input, test_labels = next(iter(test_dataloader))
print('Test Accuracy: ', getAccuracy(model, test_input, test_labels))

## Matriz de confusão com dados de treinamento e teste

In [ ]:
print('Matriz de confusão (Treino):')
display(pd.crosstab(predict(model, input_data), targets_data.numpy()))

print('Matriz de confusão (Teste):')
display(pd.crosstab(predict(model, test_input), test_labels.numpy()))

## Conclusões sobre os experimentos deste notebook
